In [ ]:
#DAVIS frames
%matplotlib inline
from matplotlib import pyplot as plt
import imageio
from scipy.io import loadmat
import cv2
import numpy as np
import matplotlib.patches as patches
import Image
import glob
#directory where the mat files are stored
DAVIS_path='/media/yy/My Passport/lipread_data/correlated_spikegrams/'
name_list= glob.glob('/media/yy/My Passport/lipread_data/correlated_spikegrams/*_normed.mat')
#print out some info about a file
print(name_list[0])
print(name_list[0][-30:-21])
vid = loadmat(name_list[0])
print(vid)

In [ ]:
#load .mat files and convert to npy files

for filename in name_list:

    vid = loadmat(filename)
    if len(vid)<4:
        continue
        
    frames=np.asarray(vid['DVS_normed'])
        out_data=frames
  
    output_folder = DAVIS_path+'np_data/'

    np.save(output_folder+filename[-30:-21]+'_dvs.npy', out_data)



In [ ]:
#cochlea
%matplotlib inline
from matplotlib import pyplot as plt
import imageio
from scipy.io import loadmat
import cv2
import numpy as np
import matplotlib.patches as patches
import Image
import glob

#directory where the mat files are stored
coch_path='/media/yy/Seagate Expansion Drive1/hw_grid_mat/DVS_frames_40_on-off/'
name_list= glob.glob('/media/yy/Seagate Expansion Drive1/DVS_frames_40_norm_part4/*_normed.mat')

#print out some info about a file
print(name_list[2000])
print(name_list[2000][-31:-21])
aud = loadmat(name_list[0])
print(aud)

In [ ]:
#load .mat files and convert to npy files
for filename in name_list:
    try:
        aud = loadmat(filename)
    except Exception as e:
        print(e)
        
    if len(aud)<3:
        continue
    if filename[-15]=='/':
        continue
        
        
    vect=np.asarray(aud['cochvector'])

    out_data=vect
    #out_data=frames.reshape((len(vid['allTsd_frames']),48,48,1))
    '''plt.figure()
    plt.subplot(2,2,1)
    plt.imshow(out_data[1],cmap='Greys_r')
    plt.subplot(2,2,2)
    plt.imshow(out_data[7],cmap='Greys_r')
    plt.subplot(2,2,3)
    plt.imshow(out_data[15],cmap='Greys_r')
    plt.subplot(2,2,4)
    plt.imshow(out_data[20],cmap='Greys_r')'''
    output_folder = coch_path+'np_data/'

    np.save(output_folder+filename[-31:-21]+'_coch.npy', out_data)


In [ ]:
#create hdf5 file 
import glob
import numpy as np

from matplotlib import pyplot as plt
%matplotlib inline
#directory that stores the npy files
VID_DATA_DIR = '/media/yy/My Passport/lipread_data/correlated_spikegrams/np_data/'
#directory that stores the key files, providing the name of files to be loaded
# the key files are used to synchronization between different type of recordings
KEY_DIR = '/media/yy/My Passport/lipread_data/correlated_spikegrams/np_data/'
np.random.seed(26)
subset_extract_ratio=0.1
train_list, test_list = [], []
list_of_vids = glob.glob(KEY_DIR+'*.npy')
for vid in list_of_vids:
    key = vid.split('/')[-1].split('_')[0]
    if np.random.random() < subset_extract_ratio:
        test_list.append(key)
    else:
        train_list.append(key)
        
print('{} keys for training, {} keys for test.'.format(
        len(train_list), len(test_list)))

from lipreading_utils import all_vocab, word_to_idx, idx_to_word, vocab_size
print('Done.')
print(test_list[0])

In [ ]:
import h5py
f = h5py.File('/media/yy/My Passport/lipread_data/current_used_datasets/DVS_corr.hdf5', 'w')
#train_aud_grp = f.create_group("train_aud")
#test_aud_grp  = f.create_group("test_aud")
#train_coch_grp = f.create_group("train_coch")
#test_coch_grp  = f.create_group("test_coch")
train_vid_grp = f.create_group("train_dvs")
test_vid_grp  = f.create_group("test_dvs")

In [ ]:
from __future__ import print_function
final_size = (48, 48)
train_list_new=[]
test_list_new=[]
miss_cnt=0
tic=0
for key in test_list:
    tic=tic+1
    if(tic%1000==0):
        print('.', end="")
    try:
        vid_data = np.load(VID_DATA_DIR+key+'_dvs.npy')
    except Exception as e:
        miss_cnt+=1
        continue

    test_list_new.append(key)
    test_vid_grp.create_dataset(key, data=vid_data)
    
tic=0
for key in train_list:
    tic=tic+1
    if(tic%1000==0):
        print('.', end="") 
    try:
        vid_data = np.load(VID_DATA_DIR+key+'_dvs.npy')
    except Exception as e:
        miss_cnt+=1
        continue
    train_list_new.append(key)
    train_vid_grp.create_dataset(key, data=vid_data)
    

train_labels  = f.create_dataset("train_labels", data=np.array(train_list_new))
test_labels   = f.create_dataset("test_labels", data=np.array(test_list_new))    
f.close()
print(miss_cnt)
print('Completed')

In [ ]:
#compute mean and std sample. Dont need this procedure for event data
f = h5py.File('/media/yy/My Passport/lipread_data/grid_subsetII/Grid_video.hdf5', "r")
# Audio
#mean_aud   = np.zeros(64)
#std_aud    = np.zeros(64)
#sum_sq_aud = np.zeros(64)
#tot_audio  = 0
# Video
mean_vid   = np.zeros((48,48))
std_vid    = np.zeros((48,48))
sum_sq_vid = np.zeros((48,48))
tot_video  = 0

# Get means
for l_idx, label in enumerate(f['train_labels']):
    # Audio
    #aud_data   = np.array(f['train_coch'][label])
    #print(aud_data.shape)
    #print(label)
    #mean_aud  += np.sum(aud_data, axis=1)
    #tot_audio += aud_data.shape[1]
    # Video
    vid_data   = np.array(f['train_vid'][label])
    mean_vid  += np.squeeze(np.sum(np.mean(vid_data, axis=3), axis=0))
    #mean_vid  += np.squeeze(np.sum(vid_data, axis=0))
    tot_video += vid_data.shape[0]
    if (l_idx+1)%1000 == 0:
        print('.'),        
mean_vid /= tot_video
#mean_aud /= tot_audio
print('Done with means.')
# Get std

for l_idx, label in enumerate(f['train_labels']):
    # Audio
    #aud_data   = np.array(f['train_coch'][label])
    #sum_sq_aud+= np.sum((np.transpose(aud_data)-mean_aud)**2, axis=0)
    # Video
    vid_data   = np.array(f['train_vid'][label])
    sum_sq_vid+= np.sum((np.mean(vid_data, axis=3)-mean_vid)**2, axis=0)
    if (l_idx+1)%1000 == 0:
        print('.'), 
print('Done with stds.')
#std_aud = np.sqrt(sum_sq_aud / tot_video)
std_vid = np.sqrt(sum_sq_vid / tot_video)

f.close()

plt.figure(figsize=(16,7))
plt.subplot(1,2,1)
plt.imshow(mean_vid, interpolation='nearest', cmap='gray')
plt.colorbar()

#plt.subplot(1,2,2)
#plt.plot(mean_aud)
#plt.grid(which='both')

plt.figure(figsize=(16,7))
plt.subplot(1,2,1)
plt.imshow(std_vid, interpolation='nearest', cmap='gray')
plt.colorbar()

#plt.subplot(1,2,2)
#plt.plot(std_aud)
#plt.grid(which='both')

plt.figure(figsize=(16,7))
plt.subplot(1,2,1)
vid_img = vid_data[0]
plt.imshow((vid_img-mean_vid)/std_vid, interpolation='nearest', cmap='gray')
plt.colorbar()
'''
plt.subplot(1,2,2)
curr_aud = aud_data[0]
plt.plot((np.transpose(curr_aud)-mean_aud)/std_aud)
plt.grid(which='both')'''